In [1]:
import numpy as np

from asteroloc8.prior import NuPrior

In [2]:
tic261154649 = NuPrior(teff_spec=4947., teff_spec_err=92., logg_spec=2.4565, logg_spec_err=0.05, plx=1.44, plx_err=0.0, 
    jmag=8.293, jmag_err=0.03, hmag=7.694, hmag_err=0.023, kmag=7.559, kmag_err=0.027)

In [3]:
a=tic261154649.gaiascalnmx(mass='giants')
print( a[0][1])

76.39175227545971


/Users/ajl573/Repos/alexlyttle/asteroloc8/asteroloc8/asteroestimate/detections/locate.py:93: RuntimeWarning: invalid value encountered in log10
  MK = Kmag-(5*np.log10(1000/parallax)-5)


In [4]:
print(tic261154649.specnmx())

((34.75368403540152, 4.264047002259701), array([36.8493626 , 41.12344521, 37.55742461, 46.2636569 , 36.42908591,
       34.31294196, 36.79221289, 35.46752081, 29.16444905, 38.3992875 ,
       40.55630813, 36.50714383, 34.29832051, 33.37472076, 39.83371691,
       36.86594723, 38.95181992, 30.57372244, 29.40991667, 41.84725541,
       31.48308072, 32.18961496, 47.04257502, 33.73304434, 29.82811071,
       35.04466731, 39.10206188, 30.92949226, 37.87620416, 35.15007766,
       31.95102728, 34.80018215, 30.16267236, 35.71455053, 32.92950634,
       38.87271621, 32.0736001 , 42.83839299, 34.01604976, 33.69346949,
       38.55460546, 34.28685412, 36.4159792 , 28.509179  , 33.76351958,
       36.80213626, 30.5841484 , 32.79535871, 37.6363662 , 36.1394119 ,
       31.42128144, 34.77585804, 36.24747445, 49.09275689, 26.07737309,
       33.21906531, 33.84543389, 35.66341938, 29.71925556, 28.06019291,
       31.26902102, 40.64824058, 42.7070766 , 36.52481104, 34.61279393,
       29.82094466, 46.

In [5]:
infile='DR16_APOTIC_NS_TMshort2.txt'

In [6]:
outfile = infile.replace('.txt','_out.txt')

In [7]:
stars=np.loadtxt(infile, usecols=(0,3,4,5,6,7,8,9,10,11,12,13,14), skiprows=1)

In [8]:
ids,ateff,atefferr,alogg,aloggerr, gparallax, gparallaxerr, tj, tjerr,th, therr,tk, tkerr=zip(*stars) 

In [9]:
open(outfile,'w').write('## TICID  specnumax  specnumaxerr gaianumax  gaianumaxerr \n')

59

In [10]:
for i in range(len(ids)):
    star=NuPrior(teff_spec=ateff[i], teff_spec_err=atefferr[i], logg_spec=alogg[i], logg_spec_err=aloggerr[i], plx=gparallax[i], plx_err=gparallaxerr[i], 
        jmag=tj[i], jmag_err=tjerr[i], hmag=th[i], hmag_err=therr[i], kmag=tk[i], kmag_err=tkerr[i])
 
    if gparallax[i] > 0:
        gaiastar=star.gaiascalnmx(mass='giants')
        gaianumaxstar=gaiastar[0][0]
        gaianumaxrangestar=gaiastar[0][1]
    else:
        gaianumaxstar=-9999
        gaianumaxrangestar=-9999

    if (ateff[i] > 0 and alogg[i] > -10):
        specstar=star.specnmx()
        specnumaxstar=specstar[0][0]
        specnumaxrangestar=specstar[0][1]
    else:
        specnumaxstar=-9999
        specnumaxrangestar=-9999   
        
    lin='   %.0f'%ids[i]+'\t'+'   %.6f'%specnumaxstar+'\t'+'   %.6f'%specnumaxrangestar+'\t'+'   %.6f'%gaianumaxstar+'\t'+'   %.6f'%gaianumaxrangestar+'\n'
    open(outfile,'a').write(lin)     